# Conversion

In [1]:
import tensorflow as tf
from qkeras import QActivation, QDense, QConv2D, QBatchNormalization, QConv2DBatchnorm
from qkeras import quantized_relu, quantized_bits

model_path = '2A_AE_model_V9_BESTOFLONGRUN'
custom_objects = {
    'QDense': QDense,
    'QActivation': QActivation,
    'QBatchNormalization': QBatchNormalization
}
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
model.summary()

2024-10-04 04:28:11.840564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 04:28:22.331256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "simplified_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 44)]              0         
                                                                 
 dense1 (QDense)             (None, 32)                1440      
                                                                 
 BN1 (QBatchNormalization)   (None, 32)                128       
                                                                 
 relu1 (QActivation)         (None, 32)                0         
                                                                 
 dense2 (QDense)             (None, 16)                528       
                                             

In [2]:
import sys
import os

local_module_dir = os.path.join("/home/jovyan/work/hls4ml") # This should be changed the path of the local modified version of HLS4ML containing Vitis Accel backend
sys.path.insert(0, local_module_dir)

import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(model, granularity="name")

hls_config['Model']['ReuseFactor'] = 1

for layer in hls_config['LayerName'].keys():
    print(layer)
    hls_config['LayerName'][layer]['Trace'] = True
    hls_config['LayerName'][layer]['Strategy'] = 'Latency'

    if layer == 'inputs':
        hls_config['LayerName'][layer]['Precision']['result'] = 'ap_fixed<15,6>'
    if layer == 'dense1':
        hls_config['LayerName'][layer]['Precision']['weight'] = 'ap_fixed<11,0>'
        hls_config['LayerName'][layer]['Precision']['bias'] = 'ap_fixed<11,0>'
        hls_config['LayerName'][layer]['Precision']['result'] = 'ap_fixed<32,5>'
    if layer == "BN1":
        hls_config['LayerName'][layer]['Precision']['bias'] = 'ap_fixed<14,0>'
        hls_config['LayerName'][layer]['Precision']['result'] = 'ap_fixed<32,5>'
    if layer == 'dense2':
        hls_config['LayerName'][layer]['Precision']['weight'] = 'ap_fixed<11,0>'
        hls_config['LayerName'][layer]['Precision']['bias'] = 'ap_fixed<11,0>'
        hls_config['LayerName'][layer]['Precision']['result'] = 'ap_fixed<21,3>'
    if layer == "BN2":
        hls_config['LayerName'][layer]['Precision']['bias'] = 'ap_fixed<14,0>'
        hls_config['LayerName'][layer]['Precision']['result'] = 'ap_fixed<27,3>'
    if layer == 'z_mean':
        hls_config['LayerName'][layer]['Precision']['weight'] = 'ap_fixed<11,6>'
        hls_config['LayerName'][layer]['Precision']['bias'] = 'ap_fixed<11,6>'
        hls_config['LayerName'][layer]['Precision']['result'] = 'ap_fixed<20,1>'

/home/jovyan/work/hls4ml/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Interpreting Model
Topology:
Layer name: inputs, layer type: InputLayer, input shapes: [[None, 44]], output shape: [None, 44]
Layer name: dense1, layer type: QDense, input shapes: [[None, 44]], output shape: [None, 32]
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Layer name: BN1, layer type: QBatchNormalization, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: relu1, layer type: LeakyReLU, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense2, layer type: QDense, input shapes: [[None, 32]], output shape: [None, 16]
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Layer name: BN2, layer type: QBatchNormalization, input shapes: [[None, 16]], output shape: [None, 16]
Layer name: relu2, layer type: LeakyReLU, input shapes: [[None, 16]], o

In [4]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=hls_config,
    clock_period=25,
    output_dir='vitis_accel_prj',
    backend='VitisAccelerator',
    board='alveo-u55c',
    num_kernel=2,
    num_worker=2,
    hw_quant=False,
    batchsize=8192,
)
hls_model.compile()

Interpreting Model
Topology:
Layer name: inputs, layer type: InputLayer, input shapes: [[None, 44]], output shape: [None, 44]
Layer name: dense1, layer type: QDense, input shapes: [[None, 44]], output shape: [None, 32]
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Layer name: BN1, layer type: QBatchNormalization, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: relu1, layer type: LeakyReLU, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense2, layer type: QDense, input shapes: [[None, 32]], output shape: [None, 16]
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Unsupported quantizer: quantized_po2
Unsupported quantizer: quantized_relu_po2
Layer name: BN2, layer type: QBatchNormalization, input shapes: [[None, 16]], output shape: [None, 16]
Layer name: relu2, layer type: LeakyReLU, input shapes: [[None, 16]], o

# Test Data Validation

In [7]:
import numpy as np

X_test = np.loadtxt("vitis_accel_prj/tb_data/tb_input_features.dat", delimiter=' ', dtype=np.float32)
print(X_test.shape)

y_keras = model.predict(X_test)
print(y_keras.shape)

with open("vitis_accel_prj/tb_data/Golden_output.dat", 'r') as f:
    lines = [line.strip() for line in f]
y_csim = np.loadtxt(lines, delimiter=' ', dtype=np.float32)
print(y_csim.shape)

np.save("X_test.npy", X_test)
np.save("y_keras.npy", y_keras)
np.save("y_csim.npy", y_csim)

(10000, 44)
313/313 [==============================] - 2s 5ms/step
(10000, 3)
(10000, 3)


In [8]:
%matplotlib inline

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

print("Accuracy -- hls4ml csim v.s. keras: {}".format(accuracy_score(np.argmax(y_csim, axis=1), np.argmax(y_keras, axis=1))))

similarity_matrix = cosine_similarity(y_keras, y_csim)
diagonal_similarities = np.diagonal(similarity_matrix)
average_similarity = np.mean(diagonal_similarities)
print("Cosine Similarity -- hls4ml csim v.s. keras: {}".format(average_similarity))

# Create a DataFrame with the first 10 entries of each array
df = pd.DataFrame({
    "y_keras": [entry for entry in y_keras[:10]],
    "y_csim": [entry for entry in y_csim[:10]]
})
df.head(10)

Accuracy -- hls4ml csim v.s. keras: 0.9599
Cosine Similarity -- hls4ml csim v.s. keras: 0.962638795375824


,y_keras,y_csim
0,"[0.008644104, 0.0512352, 0.016868591]","[0.00585938, 0.0488281, 0.0107422]"
1,"[0.035419464, 0.0776062, -0.04080963]","[0.0214844, 0.151367, -0.0410156]"
2,"[0.036201477, 0.07720566, -0.042060852]","[0.0224609, 0.15332, -0.0419922]"
3,"[-0.00016021729, 0.049575806, -0.0042495728]","[-0.000976563, 0.0478516, -0.00585938]"
4,"[-0.002670288, 0.054676056, -0.006164551]","[-0.00292969, 0.0517578, -0.00683594]"
5,"[0.013534546, 0.06694412, 0.043159485]","[0.000976563, 0.0419922, 0.0546875]"
6,"[0.007850647, 0.06211853, 0.002040863]","[0.00585938, 0.0537109, -0.000976563]"
7,"[0.095588684, 0.0882225, 0.16547012]","[0.117188, 0.101563, 0.196289]"
8,"[0.017681122, 0.144207, 0.055210114]","[0.00683594, 0.128906, 0.0361328]"
9,"[0.039569855, 0.075473785, -0.04271698]","[0.0527344, 0.12207, 0.0078125]"
